In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [27]:
# RNN 동작 이해를 위한 예제
# 입력 데이터와 레이블 데이터에 대해 문자 집합 생성 (중복제거)
input_str = 'apple'
label_str = "pple!"
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [28]:
# 하이퍼파라미터 정의
# 원 - 핫 벡터 사용

input_size = vocab_size # 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [29]:
# 문자 집합에 고유한 정수 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 문자에 고유한 인덱스 부여
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [30]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [31]:
# 입력 데이터와 레이블 데이터 각 문자들을 정수로 맵핑

x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [32]:
# 파이토치 nn.RNN() 은 기본적으로 3차원 텐서를 입력 받음 / 배치 차원 추가
# 탠서 연산인 unsqueeze(0) 를 통해 해결할 수도 있음
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [33]:
# 입력 시퀸스의 각 문자들을 원-핫 벡터로 변환

x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [34]:
# 입력데이터,레이블데이터 텐서로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

# 각 텐서의 크기 
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


In [35]:
# RNN 모델 구현 / fc는 완전 연결층 즉, 출력층으로 사용
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현

    def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [36]:
# 클래스로 정의한 모델 net에 저장
net = Net(input_size, hidden_size, output_size)

In [37]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([1, 5, 5])


In [38]:
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


In [39]:
print(Y.shape)
print(Y.view(-1).shape)

'''
레이블 데이터는 (1,5)크기를 가짐
정확도를 측정할 때는 이걸 펼쳐서 계산
(5)의 크기를 가지게 됨
'''

torch.Size([1, 5])
torch.Size([5])


In [40]:
# 손실함수, 옵티마이저 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
# epoch 100번 학습

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view 를 하는 이유는 Batch 차원 제거
    loss.backward() # 기울기 계산
    optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트
    
    # 아래 세 줄은 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드
    result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 time-step 별 5차원 벡터에 대해 높은 값의 인덱스 선택
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "predict")
    